<a href="https://colab.research.google.com/github/vgrinin/diploma/blob/main/%D0%94%D0%B8%D0%BF%D0%BB%D0%BE%D0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Недавно я прошел курсы по инвестициям в фондовый рынок. Инвестиции подразумевают регулярное пополнение своего долгосрочного инвестиционного портфеля. Мои ближайшие инвестиционные цели - 10 лет.
Однако, я хочу также на небольшие суммы (5-10 % от ежемесячных вложений) заниматься автоматическим трейдингом, для чего хочу написать своего торгового робота.
Таким образом тема дипломного проекта: Робот для трейдинга на фондовом рынке.
Помимо простой тренировки сети на временных рядах, можно еще попробовать, если хватит сил и времени, добавить в данные для обучения поиск стандартных фигур на графиках (типа двойная вершина и т.д.), ориентирование бота на новости, связанные с ожиданиями инвесторов (здесь уже может понадобиться анализ самого текста - тикер акции, общее настроение новости), добавление на вход НС значений различных стандартных индикаторов (RSI и т.д), близость выплаты дивидендов, близость и ожидания выпуска финансовой отчетности.
Бот для совершения сделок должен учитывать направление тренда и ожидаемую силу тренда, чтобы не вступать в кучу мелких сделок, а выбирать только потенциально крупные.
Сомневаюсь, что все это можно успеть к дате сдачи диплома, но хотя бы часть можно попробовать.

Здравствуйте, Вячеслав.  Тему пометил. Вам нужно внимательно посмотреть лекцию по временным рядам. Если хотите использовать любые дополнительные данные, нужно будет синхронизировать с основным рядом. Стратегию вам нужно будет определить самостоятельно. Так же у нас нет уроков по интеграции с торговым терминалом  - это придется изучить самостоятельно. В целом, по подобным работам: удавалась уверенно предсказывать тренд, но будущие значения предсказывается хуже. 

Так как у меня есть брокерский счет на Тинькове, то планирую забирать временные ряды по тикерам из его api. Собственно к api в песочнице я уже подключился. Данные из него идет в следующем формате:
{'payload': {'candles': [{'c': 229.01,
'figi': 'BBG004730RP0',
'h': 229.41,
'interval': '1min',
'l': 229.01,
'o': 229.35,
'time': datetime.datetime(2019, 8, 7, 10, 0, tzinfo=tzlocal()),
'v': 15298},
{'c': 228.82,
'figi': 'BBG004730RP0',
'h': 229.05,
'interval': '1min',
'l': 228.81,
'o': 229.05,
'time': datetime.datetime(2019, 8, 7, 10, 1, tzinfo=tzlocal()),
'v': 5193},
{'c': 228.93,
'figi': 'BBG004730RP0',
'h': 229.1,
'interval': '1min',
'l': 228.81,
'o': 228.82,
'time': datetime.datetime(2019, 8, 7, 10, 2, tzinfo=tzlocal()),
'v': 7289},
То есть это стандартные отсчеты: High, Low, Open, Close, Volume. Планирую использовать для предсказания все 5 значений, надеюсь это даст алгоритму больше понимания по волатильности рынка, свечам пробития и т.д.
По поводу использования финансовых новостей для помощи алгоритму в анализе, пока вижу такой вариант - использую сайт interfax, типа такого ула https://www.interfax.ru/business/788267. Заметил, что при последовательном декременте идентификатора новости мы просто переходим к предыдущей новости агентства, вне зависимости от того к какой категории она относится. Так что придется запустить долгий неспешный цикл закачки новостей с парсингом html, фильтрацией по категории новости, а также поиском в тексте новости ключевых слов, которые помогают отнести новость к релевантной. Фильтрация по категории как мне видится делается достаточно просто. При запросе новости, сервер присылает редирект на страницу с другой категорией, нам осталось лишь прочитать урл и найти в нем вхождение строки business. Например если запросить новость https://www.interfax.ru/business/788266 то я получаю редирект на https://www.interfax.ru/russia/788266, и вижу, что категория у новости 788266 - russia, а не business. Получив такой редирект, можно уже не запрашивать страницу с новостью, и делать декремент дальше.
Думаю, что придется выбрать 1-2 финансовых инструмента (акции), сформировать на каждый из них набор ключевых слов, для фильтрации.
Например, если выбрать акции Лукойла, то список ключевых слов примерно такой: лукойл, lukoil, нефт, бензин, топлив, и т.д.

Здравствуйте, Вячеслав. Этап принят.

 Можно выбрать проверять тексты на характерные слова, либо переводить в Embedding в BOWи сравнивать по результирующему вектору. 

In [1]:
!pip install -i https://test.pypi.org/simple/ --extra-index-url=https://pypi.org/simple/ tinkoff-invest-openapi-client

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple/


In [2]:
!git clone https://github.com/Awethon/open-api-python-client.git

fatal: destination path 'open-api-python-client' already exists and is not an empty directory.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

driveFolder = '/content/drive/My Drive/neuralnetwork/diploma/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import configparser

config = configparser.ConfigParser()
config.read(driveFolder + 'app.properties')

token = config.get("DEFAULT", "tinkoff.sandbox")

In [5]:
from openapi_client import openapi
from datetime import datetime, timedelta
from pytz import timezone
import datetime

client = openapi.sandbox_api_client(token)
client.sandbox.sandbox_register_post()
client.sandbox.sandbox_clear_post()
client.sandbox.sandbox_currencies_balance_post(sandbox_set_currency_balance_request={"currency": "USD", "balance": 1000})


def set_balance():
    balance_set = client.sandbox.sandbox_currencies_balance_post({"currency": "USD", "balance": 10000})
    print("balance")
    print(balance_set)
    print()


def print_24hr_operations():
    now = datetime.now(tz=timezone('Europe/Moscow'))
    yesterday = now - timedelta(days=1)
    ops = client.operations.operations_get(_from=yesterday.isoformat(), to=now.isoformat())
    print("operations")
    print(ops)
    print()


def print_orders():
    orders = client.orders.orders_get()
    print("active orders")
    print(orders)
    print()


def make_order():
    order_response = client.orders.orders_limit_order_post(figi='BBG009S39JX6',
                                                           limit_order_request={"lots": 1,
                                                                                "operation": "Buy",
                                                                                "price": 0.01})
    print("make order")
    print(order_response)
    print()
    return order_response


# won't work in sandbox - orders are being instantly executed
def cancel_order(order_id):
    cancellation_result = client.orders.orders_cancel_post(order_id=order_id)
    print("cancel order")
    print(cancellation_result)
    print()


'''set_balance()
print_24hr_operations()
print_orders()
order_response = make_order()
print_orders()
# cancel_order(order_response.payload.order_id)
# print_orders()
'''

#stocks = client.market.market_stocks_get()
#print(stocks)


client.market.market_candles_get('BBG004730RP0', '2019-08-07T10:00:00.029721253Z', '2019-08-07T10:05:00.029721253Z', '1min')


{'payload': {'candles': [{'c': 229.01,
                          'figi': 'BBG004730RP0',
                          'h': 229.41,
                          'interval': '1min',
                          'l': 229.01,
                          'o': 229.35,
                          'time': datetime.datetime(2019, 8, 7, 10, 0, tzinfo=tzlocal()),
                          'v': 15298},
                         {'c': 228.82,
                          'figi': 'BBG004730RP0',
                          'h': 229.05,
                          'interval': '1min',
                          'l': 228.81,
                          'o': 229.05,
                          'time': datetime.datetime(2019, 8, 7, 10, 1, tzinfo=tzlocal()),
                          'v': 5193},
                         {'c': 228.93,
                          'figi': 'BBG004730RP0',
                          'h': 229.1,
                          'interval': '1min',
                          'l': 228.81,
                          '

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

In [7]:
import csv

def csvWrite(fileName, data):
  with open(fileName, 'w', newline="", encoding='utf-16') as file:
    columns = ['datetime', 'header', 'text', 'number', 'url']
    writer = csv.DictWriter(file, delimiter=",", fieldnames=columns)
    writer.writeheader()
    writer.writerows(data)

In [8]:
requestHeaders = {
'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7,gl;q=0.6',
'cache-control': 'max-age=0',
'if-modified-since': 'Fri, 4 Sep 2021 20:55:32 GMT',
'sec-ch-ua': '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"',
'sec-ch-ua-mobile': '?0',
'sec-fetch-dest': 'document',
'sec-fetch-mode': 'navigate',
'sec-fetch-site': 'none',
'sec-fetch-user': '?1',
'upgrade-insecure-requests': '1',
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'
}

def newRequest(ssn, url):
  r = ssn.get(url, allow_redirects=False)
  r.request.headers = requestHeaders
  r.encoding='windows-1251'
  return r

In [9]:
def getRow(req, page):
  soup = BeautifulSoup(req.text, features = 'html.parser')
  mainDiv = soup.find('div', {'class': 'infinitblock'})
  timeString = mainDiv.find('time')["datetime"]
  item = mainDiv.find('article', {'itemprop': 'articleBody'})
  header = item.find('h1', {'itemprop': 'headline'}).text
  return {'datetime': timeString, 'header': header, 'text': 'text', 'number': page, 'url': req.url}

def isUrlOfCategories(url, cats):
  return any(x in url for x in cats)

In [10]:
def parseBlock(baseUrl, startPage, endPage, categories, workerName='worker'):
  print(workerName, end='')
  newsOutput = []
  ssn = requests.Session() # готовим сессию, чтобы быть более похожим на браузер
  for page in range(startPage, endPage, -1):
    print('.', end='') # прогресс, чтобы было видно, что работаем
    r = newRequest(ssn, baseUrl+'/'+str(page))
    if r.status_code == 404:
      continue
    if r.status_code == 301:
      # был редирект
      location = r.headers['Location']
      if not 'http' in location: 
        location = baseUrl + location
      # попали мы на интересующую нас категорию
      if isUrlOfCategories(location, categories):
        # если после редиректа мы получили нужную категорию, то перезапросим урл из редиректа
        r = newRequest(ssn, location)
      else:
        # Был редирект на неподходящую категорию
        continue
    newsOutput.append(getRow(r, page))
    print('\r', r.url)
    print(workerName, end='')
  return newsOutput

In [11]:
baseUrl = 'https://www.interfax.ru'
categories = ['https://www.interfax.ru/business']
startPage = 788577 
endPage = startPage - 300

newsOutput = parseBlock(baseUrl, startPage, endPage, categories, 'worker 001')
csvWrite(driveFolder + str(startPage) + '_' + str(endPage) + '.csv', newsOutput)

 https://www.interfax.ru/business/788577
 https://www.interfax.ru/business/788527
 https://www.interfax.ru/business/788525
 https://www.interfax.ru/business/788522
 https://www.interfax.ru/business/788521
 https://www.interfax.ru/business/788520
 https://www.interfax.ru/business/788516
 https://www.interfax.ru/business/788515
 https://www.interfax.ru/business/788513
 https://www.interfax.ru/business/788511
 https://www.interfax.ru/business/788499
 https://www.interfax.ru/business/788488
 https://www.interfax.ru/business/788481
 https://www.interfax.ru/business/788479
 https://www.interfax.ru/business/788472
 https://www.interfax.ru/business/788471
 https://www.interfax.ru/business/788469
 https://www.interfax.ru/business/788466
 https://www.interfax.ru/business/788464
 https://www.interfax.ru/business/788462
 https://www.interfax.ru/business/788452
 https://www.interfax.ru/business/788448
 https://www.interfax.ru/business/788442
 https://www.interfax.ru/business/788440
 https://www.int

Что нужно сделать:
* (done) формировать хидеры в запросе, как у браузера
* (done) запрашивать урл без категории, а после редиректа проверять категорию на принадлежность СПИСКУ нужных категорий
* что буду делать, если сайт начнет запрашивать у меня капчу?
* (done) хранить токен в гугл драйве
* сохранять в гуглдрайве также текущую отпаршенную страницу и сам файл с новостями
* сделать многопоточную загрузку, в каждом потоке своя сессия
* сдклать парсинг самого текста новости
* оплатить github
* сохранять проект в github